# Lab 3: A Deep Dive on `group_by()` and `summarize()` in `dplyr`

The goal of this week's lab is to get yourself more familiar with `dplyr`, specifically `group_by()` and `summarize()`. Consequently, there is some overlap between this week's and last week's materials.

In [ ]:
# Install nycflights13 package
install.packages('nycflights13')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
# Load packages
library(tidyverse)
library(nycflights13)

## Lab 2 Review
- ggplot
    - Key Components:
      - Data
      - Aesthetic Mapping
      - Layer(s) of (geom)etric Objects
    - Plots Covered:
      - Scatterplot (geom_point)
      - Boxplot (geom_boxplot)
      - Barplot (geom_bar)
      - Histogram (geom_histogram)

- dplyr
    - Work Flow: Input Dataframe -> Operation -> New Output Dataframe
    - Core Functions:
      - Select
      - Filter
      - Arrange
      - Mutate
      - Summarise (equivalently, summarize)

## Grouping

Grouping is an incredibly important operation in data science. When working with a data set, you may want to calculate a metric not for every row, but rather for every unique combination, or group, in the data set.

To give a concrete example, consider a subset of columns in the `nycflights13` data set shown below.

In [ ]:
flights %>%
  select(year, month, day, dep_delay) %>%
  print()

# A tibble: 336,776 × 4
    year month   day dep_delay
   <int> <int> <int>     <dbl>
 1  2013     1     1         2
 2  2013     1     1         4
 3  2013     1     1         2
 4  2013     1     1        -1
 5  2013     1     1        -6
 6  2013     1     1        -4
 7  2013     1     1        -5
 8  2013     1     1        -3
 9  2013     1     1        -3
10  2013     1     1        -2
# ℹ 336,766 more rows


Given this data set, one could ask: what is the average departure delay for each day of the year?

Before looking at how we can code this in R, think about the usefulness of this information for NYC airport authorities.

A possible answer: this information would allow airport authorities to be more proactive about potential delay waves through proper accommodations in advance.

Now, how can we code this? First, we need to create **groups**. As we are interested in the average departure delay for each day of the year, the days themselves are the groups!

In [ ]:
# Showcasing the grouped data set
flights %>%
  group_by(year, month, day) %>%
  print()

# A tibble: 336,776 × 19
# Groups:   year, month, day [365]
    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
 1  2013     1     1      517            515         2      830            819
 2  2013     1     1      533            529         4      850            830
 3  2013     1     1      542            540         2      923            850
 4  2013     1     1      544            545        -1     1004           1022
 5  2013     1     1      554            600        -6      812            837
 6  2013     1     1      554            558        -4      740            728
 7  2013     1     1      555            600        -5      913            854
 8  2013     1     1      557            600        -3      709            723
 9  2013     1     1      557            600        -3      838            846
10  2013     1     1      558            600        -2      753        

Note that the `group_by()` function **does not modify the input data set at all**, but rather stores the groups behind the scenes for future computation. `[365]` shows you how many unique grups there are (note that 2013 is not a leap year).

Having looked at the `group_by()` function, we now see how we can calculate a summary statistic (in our case: average departure delay) *within each group*.

## Summarize
`summarize()` takes in an input data set, performs some calculations, and outputs a new data frame. It is generally used in tandem with `group_by()`.


In [ ]:
flights %>%
  group_by(year, month, day) %>%
  summarize(mean_dep_delay = mean(dep_delay, na.rm = TRUE)) %>%
  print()

`summarise()` has grouped output by 'year', 'month'. You can override using the
`.groups` argument.


# A tibble: 365 × 4
# Groups:   year, month [12]
    year month   day mean_dep_delay
   <int> <int> <int>          <dbl>
 1  2013     1     1          11.5 
 2  2013     1     2          13.9 
 3  2013     1     3          11.0 
 4  2013     1     4           8.95
 5  2013     1     5           5.73
 6  2013     1     6           7.15
 7  2013     1     7           5.42
 8  2013     1     8           2.55
 9  2013     1     9           2.28
10  2013     1    10           2.84
# ℹ 355 more rows


This new data frame contains one column for each grouping variable (in our case: year, month, day) and one column for each summary (in our case: mean_dep_delay).

Also, another important point is the use of `na.rm = TRUE`). This is important since R does not know how to numerically handle missing values. Let's look at a simple example with a vector of numbers.

In [ ]:
### Example of the use of na.rm = TRUE

example_vector <- c(5, 6, 7, NA) # a vector of numbers with NA

example_vector %>% mean() # calculating the mean of a vector with NAs gives you <NA>

example_vector %>% mean(na.rm = TRUE) # with na.rm = TRUE, R removes the NAs and calculates the mean of the remaining numbers

[1] NA

[1] 6

Now, suppose your friend comes to you and ask which month of the year is the busiest in terms of number of flights. In R, you can count the number of occurrences of something using the function `n()`.

In [ ]:
flights %>%
  group_by(month) %>%
  summarize(n = n()) %>%
  arrange(desc(n)) %>%
  print()

# A tibble: 12 × 2
   month     n
   <int> <int>
 1     7 29425
 2     8 29327
 3    10 28889
 4     3 28834
 5     5 28796
 6     4 28330
 7     6 28243
 8    12 28135
 9     9 27574
10    11 27268
11     1 27004
12     2 24951


Alternatively, you can use `count`.

In [ ]:
flights %>%
  count(month, sort = TRUE) %>%
  print()

# A tibble: 12 × 2
   month     n
   <int> <int>
 1     7 29425
 2     8 29327
 3    10 28889
 4     3 28834
 5     5 28796
 6     4 28330
 7     6 28243
 8    12 28135
 9     9 27574
10    11 27268
11     1 27004
12     2 24951


It is important to note that `n()` also counts the missing values. If you do not want to include these missing values, `sum()` and `is.na()` can be helpful.

For example, your friend comes to you again and asks that you only include flights which are not canceled (i.e., `arr_delay` is not `NA`).

In [ ]:
flights %>%
  group_by(month) %>%
  summarize(n_notcanceled = sum(!is.na(arr_delay))) %>%
  arrange(desc(n_notcanceled)) %>%
  print()

# A tibble: 12 × 2
   month n_notcanceled
   <int>         <int>
 1     8         28756
 2    10         28618
 3     7         28293
 4     5         28128
 5     3         27902
 6     4         27564
 7     6         27075
 8    12         27020
 9     9         27010
10    11         26971
11     1         26398
12     2         23611


While July sees the most number of flights, August sees the most number of non-canceled flights.

### Sum, Mean, Maximum, Minimum, Median, Quantiles, SD, IQR

With `summarize()`, you can also calculate other summary statistics such as
1. `sum()`: summation;
2. `mean()`: average;
3. `min()` and `max()`: smallest and largest;
4. `quantile()`: `quantile(x, 0.25)` will find the value of x that is greater than 25% of all values;
5. `sd()`: standard deviation;
6. `IQR()`: interquartile range, i.e., `quantile(x, 0.75)` - `quantile(x, 0.25)`.

In [ ]:
flights %>%
  group_by(year, month, day) %>%
  summarize(
    sum = sum(dep_delay, na.rm = TRUE),
    mean = mean(dep_delay, na.rm = TRUE), # mean departure delay
    max = max(dep_delay, na.rm = TRUE), # maximum departure delay
    min = min(dep_delay, na.rm = TRUE), # minimum departure delay
    median = median(dep_delay, na.rm = TRUE), # median departure delay
    q5 = quantile(dep_delay, 0.05, na.rm = TRUE), # 5-th percentile value of departure delay
    q95 = quantile(dep_delay, 0.95, na.rm = TRUE), # 95-th percemntile of departure delay
    sd = sd(dep_delay, na.rm = TRUE), # standard deviation of departure delay
    IQR = IQR(dep_delay, na.rm = TRUE), # interquartile range of departure delay
  ) %>%
  print()

`summarise()` has grouped output by 'year', 'month'. You can override using the
`.groups` argument.


# A tibble: 365 × 12
# Groups:   year, month [12]
    year month   day   sum  mean   max   min median     q5   q95    sd   IQR
   <int> <int> <int> <dbl> <dbl> <dbl> <dbl>  <dbl>  <dbl> <dbl> <dbl> <dbl>
 1  2013     1     1  9678 11.5    853   -15     -1  -8     70.1  45.3  13  
 2  2013     1     2 12958 13.9    379   -13      0  -7     85    37.2  17  
 3  2013     1     3  9933 11.0    291   -13      0  -7.85  68    31.5  15  
 4  2013     1     4  8137  8.95   288   -19     -1  -8     60    27.7  14  
 5  2013     1     5  4110  5.73   327   -16     -1  -8     41    25.7   9  
 6  2013     1     6  5940  7.15   202   -15     -1  -8     51    23.2  11.5
 7  2013     1     7  5038  5.42   366   -17     -2  -9     51.6  27.1   8  
 8  2013     1     8  2285  2.55   188   -16     -2 -10     35.3  19.3   6  
 9  2013     1     9  2042  2.28  1301   -17     -4 -10     27.2  46.9   6  
10  2013     1    10  2643  2.84  1126   -17     -4 -10     31    43.9   6  
# ℹ 355 more rows


### Extracting the First, Last, and N-th Value
We can find the first object of `x` using `first(x)`, the last object using `last(x)`, and the nth object using `nth(x, n)`. These functions are equivalent to `x[1]`, `x[length(x)]`, `x[n]`. In order to better understand these functions, consider a simple example below.

In [ ]:
x <- c(NA, 8, 7, 10) # an example vector

# both give <NA>
x[1]
first(x)

# both give 10
x[length(x)]
last(x)

# both give 7
x[3]
nth(x, 3)

# both give <NA>
x[10]
nth(x, 10)

[1] NA

[1] NA

[1] 10

[1] 10

[1] 7

[1] 7

[1] NA

[1] NA

Now, let us find the first and 900-th arrival time for each day, among non-canceled flights.

In [ ]:
# find the first, 900th, and last arrival time everyday

flights %>%
  filter(!is.na(arr_delay)) %>%
  group_by(year, month, day) %>%
  summarize(
    first_arr = first(arr_time),
    nine_hundredth_arr = nth(arr_time, 900)) %>%
  print()

`summarise()` has grouped output by 'year', 'month'. You can override using the
`.groups` argument.


# A tibble: 365 × 5
# Groups:   year, month [12]
    year month   day first_arr nine_hundredth_arr
   <int> <int> <int>     <int>              <int>
 1  2013     1     1       830                 NA
 2  2013     1     2       518                 43
 3  2013     1     3       504                434
 4  2013     1     4       505                257
 5  2013     1     5       503                 NA
 6  2013     1     6       451                 NA
 7  2013     1     7       531               2220
 8  2013     1     8       625                 NA
 9  2013     1     9       432                 NA
10  2013     1    10       426               2206
# ℹ 355 more rows


The NAs corresponds to days in which there are less than 900 non-canceled flights.

**Warning:** The column `first_arr` does not refer to the EARLIEST arrival time for each day. It simply refers to the arrival time of the flight that occurs first in the data set for each day.

What if we want to get the EARLIEST arrival time for each day? We can use the `min()` function.

In [ ]:
flights %>%
  filter(!is.na(arr_delay)) %>%
  group_by(year, month, day) %>%
  summarize(earliest_arr = min(arr_time, na.rm = TRUE)) %>%
  print()

`summarise()` has grouped output by 'year', 'month'. You can override using the
`.groups` argument.


# A tibble: 365 × 4
# Groups:   year, month [12]
    year month   day earliest_arr
   <int> <int> <int>        <int>
 1  2013     1     1            3
 2  2013     1     2            1
 3  2013     1     3            4
 4  2013     1     4            2
 5  2013     1     5            4
 6  2013     1     6            2
 7  2013     1     7            2
 8  2013     1     8           10
 9  2013     1     9           12
10  2013     1    10            2
# ℹ 355 more rows


Technically, we can also use the `first()` function, but we need to make sure that the data is already ordered by `arr_time`.

In [ ]:
flights %>%
  filter(!is.na(arr_delay)) %>%
  arrange(arr_time) %>%
  group_by(year, month, day) %>%
  summarize(earliest_arr = first(arr_time)) %>%
  print()

`summarise()` has grouped output by 'year', 'month'. You can override using the
`.groups` argument.


# A tibble: 365 × 4
# Groups:   year, month [12]
    year month   day earliest_arr
   <int> <int> <int>        <int>
 1  2013     1     1            3
 2  2013     1     2            1
 3  2013     1     3            4
 4  2013     1     4            2
 5  2013     1     5            4
 6  2013     1     6            2
 7  2013     1     7            2
 8  2013     1     8           10
 9  2013     1     9           12
10  2013     1    10            2
# ℹ 355 more rows


## EXERCISES

1. Create a density plot which shows the distribution of the number of flights that a plane takes. Note that a plane is uniquely determined by its `tailnum`. Do not include flights with `tailnum = NA`.

In [ ]:
## YOUR ANSWER HERE

2. Produce a scatterplot where each point represents an individual plane, with the number of non-canceled flights on the x-axis and the average arrival delay on the y-axis. Recall that a flight is canceled if `arr_delay` is `NA`. Again, do not include flights with `tailnum = NA`.

In [ ]:
## YOUR ANSWER HERE

3. Which carrier has the longest arrival delay time on average?

In [ ]:
## YOUR ANSWER HERE

4. Which carrier has the highest percentange of canceled flights?

In [ ]:
## YOUR ANSWER HERE

5. Let say a departure delay is *extreme* if is it longer than 60 minutes. Find the origin airport which records the highest percentage of *extreme* delays (out of all non-canceled flights).

In [ ]:
## YOUR ANSWER HERE

6. (OPTIONAL) Continue from Question 3, where we discovered the carrier with the longest arrival delay time on average.

How does the performance compare to other carriers in the same route? Consider the differences of mean arrival delay between one carrier and the others.

*(This is an open-ended question)*

In [ ]:
## YOUR ANSWER HERE